In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from collections import defaultdict

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
train_dir = '/kaggle/input/coleridgeinitiative-show-us-the-data/train/'
test_dir = '/kaggle/input/coleridgeinitiative-show-us-the-data/test/'

**Reading the training json files**

In [ ]:
def getText(filenames, Dir=train_dir):
    text_list = []
    append = text_list.append
    for filename in filenames:
        file = pd.read_json(Dir+filename+'.json')
        text = ' '.join(file['text'].values)
        append(text)
    return text_list

In [ ]:
text_list = getText(train['Id'].values)

In [ ]:
train['text'] = text_list

In [ ]:
train.head()

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

**Combining the Existing Labels**

In [ ]:
data_title = list(map(clean_text, train['dataset_title'].unique()))
data_label = list(map(clean_text, train['dataset_label'].unique()))
cleaned_label = list(train['cleaned_label'].unique())

In [ ]:
labels = set(data_title + data_label + cleaned_label)

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
sample_submission.head()

In [ ]:
subText = getText(sample_submission['Id'].values, test_dir)
sample_submission['text'] = subText

In [ ]:
mapping = dict()
for row in train.iterrows():
    cleaned_label = row[1]['cleaned_label'].strip()
    title = clean_text(row[1]['dataset_title']).strip()
    dataSetLabel = clean_text(row[1]['dataset_label']).strip()
    mapTitle = mapping.get(title, set())
    mapTitle.add(cleaned_label)
    mapping[title] = mapTitle
    mapLabel = mapping.get(dataSetLabel, set())
    mapLabel.add(cleaned_label)
    mapping[dataSetLabel] = mapLabel
    mapCleanedLabel = mapping.get(cleaned_label, set())
    mapCleanedLabel.add(cleaned_label)
    mapping[dataSetLabel] = mapLabel

In [ ]:
mapping

In [ ]:
sample_submission.head()

**Matching the string from labels to the test text**

In [ ]:
strings = []
for text in sample_submission['text'].values:
    text = clean_text(text)
    predictString = set()
    for label in labels:
        if label in text:
            for string in mapping[label.strip()]:
                predictString.add(string)
    predictString = "|".join(predictString)
    strings.append(predictString)

**Submission !!**

In [ ]:
sample_submission['PredictionString'] = strings
sample_submission.drop(['text'], axis=1).to_csv("submission.csv", index=False)